# Big Data Systems Final Project
## Data cleaning notebook

Data source: [https://www.kaggle.com/datasets/dilwong/flightprices](https://www.kaggle.com/datasets/dilwong/flightprices)

In [43]:
import numpy as np
import pandas as pd
from isodate import parse_duration

In [2]:
data = pd.read_csv('../data/itineraries.csv', usecols=['legId',
                                                       'searchDate',
                                                       'flightDate',
                                                       'travelDuration',
                                                       'baseFare',
                                                       'seatsRemaining',
                                                       'totalTravelDistance',
                                                       'destinationAirport'
])

In [3]:
data.head()

,legId,searchDate,flightDate,destinationAirport,travelDuration,baseFare,seatsRemaining,totalTravelDistance
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,BOS,PT2H29M,217.67,9,947.0
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,BOS,PT2H30M,217.67,4,947.0
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,BOS,PT2H30M,217.67,9,947.0
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,BOS,PT2H32M,217.67,8,947.0
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,BOS,PT2H34M,217.67,9,947.0


In [4]:
data.shape

(82138753, 8)

### Dtypes and type management

In [5]:
data.dtypes

legId                   object
searchDate              object
flightDate              object
destinationAirport      object
travelDuration          object
baseFare               float64
seatsRemaining           int64
totalTravelDistance    float64
dtype: object

### Convert date columns to date so we can filter

In [12]:
data.searchDate = data.searchDate.astype('str')
data.flightDate = data.flightDate.astype('str')
data.searchDate = pd.to_datetime(data.searchDate)
data.flightDate = pd.to_datetime(data.flightDate)

In [16]:
data.dtypes

legId                          object
searchDate             datetime64[ns]
flightDate             datetime64[ns]
destinationAirport             object
travelDuration                 object
baseFare                      float64
seatsRemaining                  int64
totalTravelDistance           float64
dtype: object

### Convert `totalTravelDistance` to int

In [ ]:
data.totalTravelDistance = data.totalTravelDistance.astype('int')

### Subset `flightDate` to June, July, August

In [19]:
summer_subset = data.query('flightDate.dt.month in [6,7,8]')

In [20]:
summer_subset.shape

(42946320, 8)

### Convert dates to strings for space

In [24]:
summer_subset.searchDate = summer_subset.searchDate.astype('str')
summer_subset.flightDate = summer_subset.flightDate.astype('str')

### Drop NA

In [21]:
summer_subset = summer_subset.dropna()

In [22]:
summer_subset.shape

(39435236, 8)

In [25]:
summer_subset.head()

,legId,searchDate,flightDate,destinationAirport,travelDuration,baseFare,seatsRemaining,totalTravelDistance
480267,222cfd6d1b0d5732602a3e82ad7730c3,2022-04-17,2022-06-01,BOS,PT4H50M,65.48,4,947.0
480269,71cf5163f5efbd007c87aeef85e0c2cc,2022-04-17,2022-06-01,BOS,PT2H37M,161.86,9,947.0
480270,141ef83862caac6be402158433b55c1f,2022-04-17,2022-06-01,BOS,PT2H41M,161.86,2,947.0
480271,5921ef14d28a822fc25eb9d7879134a9,2022-04-17,2022-06-01,BOS,PT6H3M,158.14,3,956.0
480272,3ce43c14cd65f1d4303a4b90093f328c,2022-04-17,2022-06-01,BOS,PT8H13M,151.63,9,947.0


### Clean `travelDuration`

In [27]:
summer_subset.travelDuration.head()

480267    PT4H50M
480269    PT2H37M
480270    PT2H41M
480271     PT6H3M
480272    PT8H13M
Name: travelDuration, dtype: object

In [46]:
summer_subset.travelDuration.sample(5)

28606778       PT9H
7969588     PT2H29M
37266859    PT9H50M
33444554    PT7H51M
37410829    PT5H24M
Name: travelDuration, dtype: object

In [44]:
def duration_str_to_seconds(duration_str):
    duration = parse_duration(duration_str)
    return int(duration.total_seconds())

In [45]:
# Example usage:
duration = "PT5H29M"
seconds = duration_str_to_seconds(duration)
print("Duration in seconds:", seconds)

Duration in seconds: 19740


In [48]:
summer_subset['durationSeconds'] = summer_subset.travelDuration.apply(lambda x: duration_str_to_seconds(x))

In [49]:
summer_subset.head()

,legId,searchDate,flightDate,destinationAirport,travelDuration,baseFare,seatsRemaining,totalTravelDistance,durationSeconds
480267,222cfd6d1b0d5732602a3e82ad7730c3,2022-04-17,2022-06-01,BOS,PT4H50M,65.48,4,947.0,17400
480269,71cf5163f5efbd007c87aeef85e0c2cc,2022-04-17,2022-06-01,BOS,PT2H37M,161.86,9,947.0,9420
480270,141ef83862caac6be402158433b55c1f,2022-04-17,2022-06-01,BOS,PT2H41M,161.86,2,947.0,9660
480271,5921ef14d28a822fc25eb9d7879134a9,2022-04-17,2022-06-01,BOS,PT6H3M,158.14,3,956.0,21780
480272,3ce43c14cd65f1d4303a4b90093f328c,2022-04-17,2022-06-01,BOS,PT8H13M,151.63,9,947.0,29580


### Drop `travelDuration` now that we've converted to seconds

In [50]:
summer_subset = summer_subset.drop('travelDuration', axis=1)

In [51]:
summer_subset.head()

,legId,searchDate,flightDate,destinationAirport,baseFare,seatsRemaining,totalTravelDistance,durationSeconds
480267,222cfd6d1b0d5732602a3e82ad7730c3,2022-04-17,2022-06-01,BOS,65.48,4,947.0,17400
480269,71cf5163f5efbd007c87aeef85e0c2cc,2022-04-17,2022-06-01,BOS,161.86,9,947.0,9420
480270,141ef83862caac6be402158433b55c1f,2022-04-17,2022-06-01,BOS,161.86,2,947.0,9660
480271,5921ef14d28a822fc25eb9d7879134a9,2022-04-17,2022-06-01,BOS,158.14,3,956.0,21780
480272,3ce43c14cd65f1d4303a4b90093f328c,2022-04-17,2022-06-01,BOS,151.63,9,947.0,29580


### Save `summer_subset` to .csv

In [58]:
summer_subset.to_csv('cleaned_data.csv')